# Problem 2: Multi-Layer Chain Rule - How Deep Networks Learn

## Learning Objectives
By the end of this problem, you will:
- Understand backpropagation as systematic application of the chain rule
- Analyze gradient flow through multiple computational layers
- Identify and diagnose vanishing/exploding gradient problems
- Connect mathematical chain rule to practical deep learning implementation

## Task Overview

1. **Chain Rule Fundamentals** - From single to multi-variable chain rule
2. **Multi-Layer Network Analysis** - Extend "Go Dolphins!" to deep architecture
3. **Gradient Flow Dynamics** - Track gradients through network layers
4. **Backpropagation Implementation** - Build the algorithm from mathematical principles

---

## From Single Layer to Deep Networks

In Problem 1, you analyzed the loss landscape for our single-layer "Go Dolphins!" classifier:
```
Input [2,1,1] → Weights [w₁,w₂,w₃] → Prediction → Loss
```

But modern AI systems use **deep networks** with many layers:
```
Input → Hidden Layer 1 → Hidden Layer 2 → ... → Output → Loss
```

**The Challenge**: How do we compute gradients when the loss depends on weights through a complex chain of transformations?

**The Solution**: The **chain rule** - one of the most important theorems in calculus and the mathematical foundation of deep learning.

## Mathematical Foundation: The Chain Rule

**Single Variable Chain Rule**:
If $y = f(g(x))$, then $\frac{dy}{dx} = \frac{df}{dg} \cdot \frac{dg}{dx}$

**Multi-Variable Chain Rule**:
If $z = f(x, y)$ where $x = g(t)$ and $y = h(t)$, then:
$$\frac{dz}{dt} = \frac{\partial z}{\partial x} \frac{dx}{dt} + \frac{\partial z}{\partial y} \frac{dy}{dt}$$

**Deep Network Chain Rule**:
For loss $L$ depending on weights $W^{(1)}$ through layers $z^{(1)}, z^{(2)}, ..., z^{(n)}$:
$$\frac{\partial L}{\partial W^{(1)}} = \frac{\partial L}{\partial z^{(n)}} \frac{\partial z^{(n)}}{\partial z^{(n-1)}} \cdots \frac{\partial z^{(2)}}{\partial z^{(1)}} \frac{\partial z^{(1)}}{\partial W^{(1)}}$$

This is **backpropagation** - working backwards through the network, applying the chain rule at each step!

## Why This Matters

Understanding the mathematical chain rule reveals:
- **Why backpropagation works** (systematic application of calculus)
- **Where gradients vanish** (product of many small derivatives)
- **How to design stable architectures** (controlling gradient magnitudes)
- **The computational efficiency** (reusing intermediate calculations)

Let's extend our "Go Dolphins!" example to see this in action!

In [ ]:
# Setup for multi-layer chain rule analysis
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict

# Import our utilities
import sys
sys.path.append('./utils')
from data_generators import load_sports_dataset

# Load our "Go Dolphins!" dataset
features, labels, feature_names, texts = load_sports_dataset()

print("DEEP NETWORK ANALYSIS: EXTENDING 'GO DOLPHINS!' TO MULTIPLE LAYERS")
print("=" * 70)
print(f"Dataset: {len(texts)} sports tweets")
print(f"Base features: {feature_names}")
print()
print("Network architectures we'll analyze:")
print("• Single layer: Input(3) → Output(1)")
print("• Two layer: Input(3) → Hidden(4) → Output(1)")
print("• Three layer: Input(3) → Hidden(4) → Hidden(2) → Output(1)")
print("• Deep network: Input(3) → Hidden(6) → Hidden(4) → Hidden(2) → Output(1)")
print()
print("Mathematical focus: Chain rule application through each architecture")

# Define activation functions and their derivatives
def sigmoid(x):
    """Sigmoid activation function"""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def sigmoid_derivative(x):
    """Derivative of sigmoid: σ'(x) = σ(x)(1 - σ(x))"""
    s = sigmoid(x)
    return s * (1 - s)

def relu(x):
    """ReLU activation function"""
    return np.maximum(0, x)

def relu_derivative(x):
    """Derivative of ReLU: 1 if x > 0, 0 otherwise"""
    return (x > 0).astype(float)

def tanh(x):
    """Hyperbolic tangent activation"""
    return np.tanh(x)

def tanh_derivative(x):
    """Derivative of tanh: 1 - tanh²(x)"""
    return 1 - np.tanh(x)**2

# Binary cross-entropy loss and its derivative
def bce_loss(predictions, targets):
    """Binary cross-entropy loss"""
    predictions = np.clip(predictions, 1e-15, 1 - 1e-15)
    return -np.mean(targets * np.log(predictions) + (1 - targets) * np.log(1 - predictions))

def bce_loss_derivative(predictions, targets):
    """Derivative of BCE loss with respect to predictions"""
    predictions = np.clip(predictions, 1e-15, 1 - 1e-15)
    return (predictions - targets) / (predictions * (1 - predictions)) / len(targets)

print("\n✅ Activation functions and derivatives ready for chain rule analysis!")

## Task 1: Chain Rule Fundamentals

Let's build intuition by starting with simple composite functions and working up to neural network complexity.

In [ ]:
# TODO: Demonstrate chain rule with simple examples
def demonstrate_chain_rule_basics():
    """
    Build intuition with simple chain rule examples.
    """
    print("CHAIN RULE FUNDAMENTALS")
    print("=" * 30)
    
    # Example 1: Single variable chain rule
    print("Example 1: Single Variable Chain Rule")
    print("Function: f(g(x)) where g(x) = x² + 1, f(u) = sin(u)")
    print("So h(x) = sin(x² + 1)")
    print()
    
    x = 2.0
    
    # Forward pass
    g_x = x**2 + 1  # g(x) = x² + 1
    f_g = np.sin(g_x)  # f(g(x)) = sin(g(x))
    
    # Analytical derivatives
    dg_dx = 2 * x  # g'(x) = 2x
    df_dg = np.cos(g_x)  # f'(u) = cos(u)
    
    # Chain rule: dh/dx = df/dg * dg/dx
    dh_dx_chain = df_dg * dg_dx
    
    # Verify with numerical differentiation
    h = 1e-8
    dh_dx_numerical = (np.sin((x+h)**2 + 1) - np.sin((x-h)**2 + 1)) / (2*h)
    
    print(f"At x = {x}:")
    print(f"  g(x) = {g_x}")
    print(f"  f(g(x)) = {f_g:.6f}")
    print(f"  dg/dx = {dg_dx}")
    print(f"  df/dg = {df_dg:.6f}")
    print(f"  Chain rule: dh/dx = {dh_dx_chain:.6f}")
    print(f"  Numerical: dh/dx = {dh_dx_numerical:.6f}")
    print(f"  Difference: {abs(dh_dx_chain - dh_dx_numerical):.2e}")
    print()
    
    # Example 2: Neural network style composition
    print("Example 2: Neural Network Style Composition")
    print("Function: sigmoid(w·x + b) where x = [2, 1, 1], w = [0.3, 0.5, 0.4], b = 0.1")
    print()
    
    x = np.array([2, 1, 1])
    w = np.array([0.3, 0.5, 0.4])
    b = 0.1
    
    # Forward pass
    z = np.dot(w, x) + b  # Linear transformation
    a = sigmoid(z)  # Activation
    
    # Chain rule for ∂a/∂w
    da_dz = sigmoid_derivative(z)  # ∂σ/∂z
    dz_dw = x  # ∂z/∂w = x (since z = w·x + b)
    da_dw = da_dz * dz_dw  # Chain rule
    
    print(f"Input x: {x}")
    print(f"Weights w: {w}")
    print(f"Bias b: {b}")
    print(f"Linear output z: {z:.6f}")
    print(f"Activation output a: {a:.6f}")
    print(f"∂a/∂z: {da_dz:.6f}")
    print(f"∂z/∂w: {dz_dw}")
    print(f"∂a/∂w (chain rule): {da_dw}")
    
    # Verify numerically for first weight
    h = 1e-8
    w_plus = w.copy()
    w_minus = w.copy()
    w_plus[0] += h
    w_minus[0] -= h
    
    a_plus = sigmoid(np.dot(w_plus, x) + b)
    a_minus = sigmoid(np.dot(w_minus, x) + b)
    da_dw0_numerical = (a_plus - a_minus) / (2*h)
    
    print(f"\nVerification for ∂a/∂w₀:")
    print(f"  Chain rule: {da_dw[0]:.8f}")
    print(f"  Numerical:  {da_dw0_numerical:.8f}")
    print(f"  Difference: {abs(da_dw[0] - da_dw0_numerical):.2e}")

demonstrate_chain_rule_basics()
print("\n✅ Chain rule fundamentals demonstrated!")

In [ ]:
# TODO: Analyze gradient magnitude through compositions
def analyze_gradient_magnitudes():
    """
    Explore how gradient magnitudes change through function compositions.
    """
    print("\nGRADIENT MAGNITUDE ANALYSIS THROUGH COMPOSITIONS")
    print("=" * 55)
    
    # Test different activation functions in chain
    x_vals = np.linspace(-3, 3, 100)
    
    activations = {
        'sigmoid': (sigmoid, sigmoid_derivative),
        'tanh': (tanh, tanh_derivative),
        'relu': (relu, relu_derivative)
    }
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    for i, (name, (func, deriv)) in enumerate(activations.items()):
        # Plot activation function
        y_vals = func(x_vals)
        axes[0, i].plot(x_vals, y_vals, 'b-', linewidth=2, label=f'{name}(x)')
        axes[0, i].set_xlabel('x')
        axes[0, i].set_ylabel('f(x)')
        axes[0, i].set_title(f'{name.capitalize()} Activation')
        axes[0, i].grid(True, alpha=0.3)
        axes[0, i].legend()
        
        # Plot derivative
        dy_vals = deriv(x_vals)
        axes[1, i].plot(x_vals, dy_vals, 'r-', linewidth=2, label=f"{name}'(x)")
        axes[1, i].set_xlabel('x')
        axes[1, i].set_ylabel("f'(x)")
        axes[1, i].set_title(f'{name.capitalize()} Derivative')
        axes[1, i].grid(True, alpha=0.3)
        axes[1, i].legend()
        
        # Analyze derivative properties
        max_deriv = np.max(dy_vals)
        min_deriv = np.min(dy_vals)
        print(f"\n{name.capitalize()} activation:")
        print(f"  Max derivative: {max_deriv:.4f}")
        print(f"  Min derivative: {min_deriv:.4f}")
        print(f"  Range: [{min_deriv:.4f}, {max_deriv:.4f}]")
        
        if name == 'sigmoid':
            print(f"  ⚠️  Max derivative < 1: potential vanishing gradients")
        elif name == 'relu':
            print(f"  ✅ Derivative is 0 or 1: no vanishing (but can have dying ReLU)")
    
    plt.tight_layout()
    plt.show()
    
    # Demonstrate gradient flow through multiple layers
    print("\nGRADIENT FLOW THROUGH MULTIPLE LAYERS:")
    print("=" * 45)
    
    # Simulate gradient flowing backwards through layers
    initial_gradient = 1.0  # Start with unit gradient
    layer_inputs = [-2, -1, 0, 1, 2]  # Different activation regions
    
    for activation_name, (func, deriv) in activations.items():
        print(f"\n{activation_name.upper()} through 3 layers:")
        print(f"{'Layer':<8} | {'Input':<8} | {'Derivative':<12} | {'Gradient':<12}")
        print("-" * 50)
        
        gradient = initial_gradient
        for layer in range(3):
            # Use different inputs for each layer
            layer_input = layer_inputs[layer % len(layer_inputs)]
            layer_deriv = deriv(layer_input)
            gradient *= layer_deriv
            
            print(f"Layer {layer+1:<3} | {layer_input:<8.1f} | {layer_deriv:<12.6f} | {gradient:<12.6f}")
        
        print(f"Final gradient magnitude: {abs(gradient):.6f}")
        if abs(gradient) < 0.1:
            print("⚠️  Vanishing gradient detected!")
        elif abs(gradient) > 10:
            print("⚠️  Exploding gradient detected!")
        else:
            print("✅ Gradient magnitude reasonable")

analyze_gradient_magnitudes()
print("\n✅ Gradient magnitude analysis complete!")

## Task 2: Multi-Layer Network Analysis

Now let's extend our "Go Dolphins!" classifier to multiple layers and analyze gradient flow through the architecture.

In [ ]:
# TODO: Implement multi-layer neural network
class MultiLayerNetwork:
    """
    Multi-layer neural network for mathematical analysis.
    """
    
    def __init__(self, layer_sizes, activation='sigmoid'):
        """
        Initialize network with specified architecture.
        
        Args:
            layer_sizes: List of layer sizes [input, hidden1, hidden2, ..., output]
            activation: Activation function ('sigmoid', 'tanh', 'relu')
        """
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes) - 1
        self.activation = activation
        
        # Initialize weights and biases
        self.weights = []
        self.biases = []
        
        for i in range(self.num_layers):
            # Xavier/Glorot initialization
            scale = np.sqrt(2.0 / (layer_sizes[i] + layer_sizes[i+1]))
            weight_matrix = np.random.normal(0, scale, (layer_sizes[i], layer_sizes[i+1]))
            bias_vector = np.zeros(layer_sizes[i+1])
            
            self.weights.append(weight_matrix)
            self.biases.append(bias_vector)
        
        # Store activation functions
        if activation == 'sigmoid':
            self.activation_func = sigmoid
            self.activation_deriv = sigmoid_derivative
        elif activation == 'tanh':
            self.activation_func = tanh
            self.activation_deriv = tanh_derivative
        elif activation == 'relu':
            self.activation_func = relu
            self.activation_deriv = relu_derivative
        
        print(f"Initialized {self.num_layers}-layer network: {layer_sizes}")
        print(f"Activation: {activation}")
        print(f"Total parameters: {self.count_parameters()}")
    
    def count_parameters(self):
        """Count total number of parameters."""
        total = 0
        for w, b in zip(self.weights, self.biases):
            total += w.size + b.size
        return total
    
    def forward_pass(self, x, store_activations=True):
        """
        Forward pass through the network.
        
        Returns:
            output: Final network output
            activations: List of activations at each layer (if store_activations=True)
            z_values: List of pre-activation values (if store_activations=True)
        """
        if store_activations:
            activations = [x]  # Input is first activation
            z_values = []  # Pre-activation values
        
        current_input = x
        
        for i in range(self.num_layers):
            # Linear transformation: z = W^T * a + b
            z = np.dot(current_input, self.weights[i]) + self.biases[i]
            
            if store_activations:
                z_values.append(z)
            
            # Apply activation (except for output layer - we'll use sigmoid for final)
            if i == self.num_layers - 1:
                # Output layer always uses sigmoid for binary classification
                a = sigmoid(z)
            else:
                # Hidden layers use specified activation
                a = self.activation_func(z)
            
            if store_activations:
                activations.append(a)
            
            current_input = a
        
        if store_activations:
            return current_input, activations, z_values
        else:
            return current_input
    
    def backward_pass(self, x, y, activations, z_values):
        """
        Backward pass using chain rule (backpropagation).
        
        Returns:
            gradients_w: List of weight gradients
            gradients_b: List of bias gradients
            gradient_magnitudes: Gradient magnitudes at each layer
        """
        gradients_w = [np.zeros_like(w) for w in self.weights]
        gradients_b = [np.zeros_like(b) for b in self.biases]
        gradient_magnitudes = []
        
        # Start with loss gradient (BCE with sigmoid output)
        output = activations[-1]
        delta = output - y  # For BCE + sigmoid, this is the gradient
        
        # Backward through each layer
        for i in reversed(range(self.num_layers)):
            # Record gradient magnitude for analysis
            gradient_magnitudes.append(np.linalg.norm(delta))
            
            # Gradients for current layer
            gradients_w[i] = np.outer(activations[i], delta)
            gradients_b[i] = delta.copy()
            
            # Propagate gradient to previous layer (if not input layer)
            if i > 0:
                # Chain rule: δ^(l-1) = (W^(l))^T δ^(l) ⊙ σ'(z^(l-1))
                delta = np.dot(self.weights[i], delta)
                
                # Apply activation derivative
                if i == 1:  # Previous layer is first hidden layer
                    delta *= self.activation_deriv(z_values[i-1])
                else:  # Previous layer is another hidden layer
                    delta *= self.activation_deriv(z_values[i-1])
        
        # Reverse gradient magnitudes to match layer order
        gradient_magnitudes.reverse()
        
        return gradients_w, gradients_b, gradient_magnitudes

# Test different network architectures on "Go Dolphins!"
print("MULTI-LAYER 'GO DOLPHINS!' CLASSIFIER ANALYSIS")
print("=" * 50)

# Define different architectures to test
architectures = [
    ([3, 1], "Single Layer"),
    ([3, 4, 1], "Two Layer"),
    ([3, 4, 2, 1], "Three Layer"),
    ([3, 6, 4, 2, 1], "Four Layer (Deep)")
]

# Test each architecture
networks = {}
for arch, name in architectures:
    print(f"\n{name} Network: {arch}")
    print("-" * 30)
    
    # Create network
    net = MultiLayerNetwork(arch, activation='sigmoid')
    networks[name] = net
    
    # Test forward pass on "Go Dolphins!" example
    x = features[0]  # "Go Dolphins!" features [2, 1, 1]
    y = labels[0]    # True label (positive)
    
    output, activations, z_values = net.forward_pass(x)
    
    print(f"Input: {x}")
    print(f"Output: {output:.6f}")
    print(f"Target: {y}")
    print(f"Loss: {bce_loss(np.array([output]), np.array([y])):.6f}")
    
    # Show activations at each layer
    for i, activation in enumerate(activations):
        if i == 0:
            print(f"Layer {i} (input): {activation}")
        elif i == len(activations) - 1:
            print(f"Layer {i} (output): {activation:.6f}")
        else:
            print(f"Layer {i} (hidden): {activation}")

print("\n✅ Multi-layer networks initialized and tested!")

## Task 3: Gradient Flow Dynamics

Let's analyze how gradients flow backwards through our multi-layer networks and identify potential problems.

In [ ]:
# TODO: Analyze gradient flow through different architectures
def analyze_gradient_flow():
    """
    Analyze gradient flow through different network architectures.
    """
    print("GRADIENT FLOW ANALYSIS")
    print("=" * 30)
    
    # Test on a single example for detailed analysis
    x = features[0]  # "Go Dolphins!" example
    y = labels[0]    # True label
    
    gradient_data = {}
    
    for name, net in networks.items():
        print(f"\n{name} Network Gradient Flow:")
        print("-" * 35)
        
        # Forward pass
        output, activations, z_values = net.forward_pass(x)
        
        # Backward pass
        grad_w, grad_b, grad_mags = net.backward_pass(x, y, activations, z_values)
        
        gradient_data[name] = {
            'magnitudes': grad_mags,
            'num_layers': net.num_layers,
            'activations': activations,
            'z_values': z_values
        }
        
        print(f"Forward pass output: {output:.6f}")
        print(f"Target: {y}")
        print(f"Number of layers: {net.num_layers}")
        print("\nGradient magnitudes by layer (output → input):")
        
        for i, mag in enumerate(grad_mags):
            layer_name = f"Layer {net.num_layers - i}"
            if i == 0:
                layer_name += " (output)"
            elif i == len(grad_mags) - 1:
                layer_name += " (first hidden)"
            else:
                layer_name += " (hidden)"
            
            print(f"  {layer_name}: {mag:.6f}")
        
        # Calculate gradient decay ratio
        if len(grad_mags) > 1:
            decay_ratio = grad_mags[-1] / grad_mags[0]  # First layer / output layer
            print(f"\nGradient decay ratio (first/output): {decay_ratio:.6f}")
            
            if decay_ratio < 0.01:
                print("⚠️  Severe vanishing gradient problem!")
            elif decay_ratio < 0.1:
                print("⚠️  Moderate vanishing gradient problem")
            elif decay_ratio > 10:
                print("⚠️  Exploding gradient problem!")
            else:
                print("✅ Gradient flow looks healthy")
        
        # Analyze activation saturation
        print("\nActivation analysis:")
        for i, (activation, z) in enumerate(zip(activations[1:], z_values)):
            layer_num = i + 1
            if isinstance(activation, np.ndarray):
                avg_activation = np.mean(activation)
                avg_z = np.mean(z)
            else:
                avg_activation = activation
                avg_z = z
            
            print(f"  Layer {layer_num}: avg_z = {avg_z:.4f}, avg_activation = {avg_activation:.4f}")
            
            # Check for saturation in sigmoid
            if net.activation == 'sigmoid' and layer_num < net.num_layers:
                if abs(avg_z) > 3:
                    print(f"    ⚠️  Saturated sigmoid (|z| = {abs(avg_z):.2f} > 3)")
                else:
                    print(f"    ✅ Good sigmoid range (|z| = {abs(avg_z):.2f} < 3)")
    
    return gradient_data

gradient_analysis = analyze_gradient_flow()
print("\n✅ Gradient flow analysis complete!")

In [ ]:
# TODO: Visualize gradient flow patterns
def visualize_gradient_flow(gradient_data):
    """
    Create visualizations of gradient flow through networks.
    """
    print("\nGRADIENT FLOW VISUALIZATION")
    print("=" * 35)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    
    # Plot 1: Gradient magnitude decay
    ax = axes[0]
    for name, data in gradient_data.items():
        layers = list(range(data['num_layers'], 0, -1))  # Output to input
        magnitudes = data['magnitudes']
        ax.plot(layers, magnitudes, 'o-', linewidth=2, markersize=8, label=name)
    
    ax.set_xlabel('Layer Number (Output → Input)')
    ax.set_ylabel('Gradient Magnitude')
    ax.set_title('Gradient Magnitude by Layer')
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Gradient decay ratio
    ax = axes[1]
    network_names = []
    decay_ratios = []
    
    for name, data in gradient_data.items():
        if len(data['magnitudes']) > 1:
            ratio = data['magnitudes'][-1] / data['magnitudes'][0]
            network_names.append(name)
            decay_ratios.append(ratio)
    
    bars = ax.bar(network_names, decay_ratios, alpha=0.7)
    ax.set_ylabel('Gradient Decay Ratio')
    ax.set_title('Gradient Decay: First Layer / Output Layer')
    ax.set_yscale('log')
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3)
    
    # Color bars based on severity
    for bar, ratio in zip(bars, decay_ratios):
        if ratio < 0.01:
            bar.set_color('red')  # Severe vanishing
        elif ratio < 0.1:
            bar.set_color('orange')  # Moderate vanishing
        elif ratio > 10:
            bar.set_color('purple')  # Exploding
        else:
            bar.set_color('green')  # Healthy
    
    # Plot 3: Activation distributions
    ax = axes[2]
    
    # Compare different network depths for the deepest network
    deepest_net_name = max(gradient_data.keys(), key=lambda k: gradient_data[k]['num_layers'])
    deepest_data = gradient_data[deepest_net_name]
    
    for i, activation in enumerate(deepest_data['activations'][1:], 1):  # Skip input
        if isinstance(activation, np.ndarray) and len(activation) > 1:
            ax.hist(activation, bins=10, alpha=0.6, label=f'Layer {i}', density=True)
        else:
            # Single value - show as vertical line
            ax.axvline(activation, alpha=0.8, linewidth=3, label=f'Layer {i}')
    
    ax.set_xlabel('Activation Value')
    ax.set_ylabel('Density')
    ax.set_title(f'Activation Distributions ({deepest_net_name})')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Chain rule multiplication effect
    ax = axes[3]
    
    # Simulate cumulative gradient multiplication
    x_vals = np.linspace(-3, 3, 100)
    sigmoid_deriv_vals = sigmoid_derivative(x_vals)
    
    cumulative_products = []
    for num_layers in [1, 2, 3, 4, 5]:
        products = sigmoid_deriv_vals ** num_layers
        cumulative_products.append(products)
        ax.plot(x_vals, products, linewidth=2, label=f'{num_layers} layers')
    
    ax.set_xlabel('Pre-activation Value (z)')
    ax.set_ylabel('Cumulative Derivative Product')
    ax.set_title('Chain Rule Effect: σ\'(z)^n')
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Analysis summary
    print("\nKEY INSIGHTS FROM VISUALIZATION:")
    print("1. Plot 1 shows how gradient magnitude decreases through layers")
    print("2. Plot 2 quantifies the vanishing gradient problem")
    print("3. Plot 3 reveals activation saturation patterns")
    print("4. Plot 4 demonstrates why deep networks struggle with sigmoid")

visualize_gradient_flow(gradient_analysis)
print("\n✅ Gradient flow visualization complete!")

## Task 4: Backpropagation Implementation

Let's implement backpropagation from scratch, showing exactly how the chain rule is applied systematically.

In [ ]:
# TODO: Implement detailed backpropagation with step-by-step chain rule
class DetailedBackpropNetwork:
    """
    Network that shows detailed backpropagation steps for educational purposes.
    """
    
    def __init__(self, layer_sizes):
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes) - 1
        
        # Initialize weights and biases
        self.weights = []
        self.biases = []
        
        np.random.seed(42)  # For reproducible analysis
        for i in range(self.num_layers):
            w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.5
            b = np.zeros(layer_sizes[i+1])
            self.weights.append(w)
            self.biases.append(b)
    
    def detailed_forward_pass(self, x, verbose=True):
        """
        Forward pass with detailed intermediate steps.
        """
        if verbose:
            print("DETAILED FORWARD PASS")
            print("=" * 25)
        
        activations = [x]
        z_values = []
        
        current_input = x
        if verbose:
            print(f"Input: {current_input}")
        
        for i in range(self.num_layers):
            if verbose:
                print(f"\nLayer {i+1}:")
                print(f"  Weights shape: {self.weights[i].shape}")
                print(f"  Weights:\n{self.weights[i]}")
                print(f"  Biases: {self.biases[i]}")
            
            # Linear transformation
            z = np.dot(current_input, self.weights[i]) + self.biases[i]
            z_values.append(z)
            
            if verbose:
                print(f"  Pre-activation (z): {z}")
            
            # Activation
            a = sigmoid(z)
            activations.append(a)
            
            if verbose:
                print(f"  Post-activation (a): {a}")
            
            current_input = a
        
        return current_input, activations, z_values
    
    def detailed_backward_pass(self, x, y, activations, z_values, verbose=True):
        """
        Backward pass with detailed chain rule steps.
        """
        if verbose:
            print("\nDETAILED BACKWARD PASS (CHAIN RULE APPLICATION)")
            print("=" * 55)
        
        # Initialize gradients
        grad_weights = [np.zeros_like(w) for w in self.weights]
        grad_biases = [np.zeros_like(b) for b in self.biases]
        
        # Start with loss gradient
        output = activations[-1]
        loss = bce_loss(np.array([output]), np.array([y]))
        
        if verbose:
            print(f"Final output: {output}")
            print(f"Target: {y}")
            print(f"Loss: {loss:.6f}")
            print(f"\nStarting backpropagation...")
        
        # For BCE + sigmoid: ∂L/∂a = (a - y) / (a(1-a))
        # But for ∂L/∂z (before sigmoid): ∂L/∂z = a - y
        delta = output - y
        
        if verbose:
            print(f"\nOutput layer gradient:")
            print(f"  ∂L/∂z_output = prediction - target = {output:.6f} - {y} = {delta:.6f}")
        
        # Backward through each layer
        for i in reversed(range(self.num_layers)):
            layer_num = i + 1
            if verbose:
                print(f"\nLayer {layer_num} gradients:")
                print(f"  Current delta (∂L/∂z): {delta}")
            
            # Gradients for weights and biases
            # ∂L/∂W = a^(l-1) ⊗ δ^(l) (outer product)
            # ∂L/∂b = δ^(l)
            prev_activation = activations[i]
            
            if isinstance(delta, np.ndarray) and isinstance(prev_activation, np.ndarray):
                if delta.ndim == 0 and prev_activation.ndim == 1:
                    grad_weights[i] = np.outer(prev_activation, delta)
                else:
                    grad_weights[i] = np.outer(prev_activation, delta)
            else:
                grad_weights[i] = np.outer(prev_activation, delta)
            
            grad_biases[i] = delta.copy() if isinstance(delta, np.ndarray) else np.array([delta])
            
            if verbose:
                print(f"  Previous activation: {prev_activation}")
                print(f"  ∂L/∂W (outer product): \n{grad_weights[i]}")
                print(f"  ∂L/∂b: {grad_biases[i]}")
            
            # Propagate delta to previous layer (if not input layer)
            if i > 0:
                # ∂L/∂a^(l-1) = W^(l) × δ^(l)
                delta_prev = np.dot(self.weights[i], delta)
                
                # ∂L/∂z^(l-1) = ∂L/∂a^(l-1) ⊙ σ'(z^(l-1))
                sigmoid_deriv = sigmoid_derivative(z_values[i-1])
                delta = delta_prev * sigmoid_deriv
                
                if verbose:
                    print(f"  Propagating to layer {i}:")
                    print(f"    ∂L/∂a_prev = W^T × δ = {self.weights[i].flatten()} × {delta} = {delta_prev}")
                    print(f"    σ'(z_prev) = {sigmoid_deriv}")
                    print(f"    ∂L/∂z_prev = ∂L/∂a_prev ⊙ σ'(z_prev) = {delta}")
        
        return grad_weights, grad_biases, loss
    
    def numerical_gradients(self, x, y, h=1e-8):
        """
        Compute numerical gradients for verification.
        """
        numerical_grad_w = []
        numerical_grad_b = []
        
        def compute_loss(weights, biases):
            # Temporarily set weights and biases
            old_weights = [w.copy() for w in self.weights]
            old_biases = [b.copy() for b in self.biases]
            
            for i in range(len(weights)):
                self.weights[i] = weights[i]
                self.biases[i] = biases[i]
            
            output, _, _ = self.detailed_forward_pass(x, verbose=False)
            loss = bce_loss(np.array([output]), np.array([y]))
            
            # Restore weights and biases
            for i in range(len(old_weights)):
                self.weights[i] = old_weights[i]
                self.biases[i] = old_biases[i]
            
            return loss
        
        # Numerical gradients for weights
        for i in range(self.num_layers):
            grad_w = np.zeros_like(self.weights[i])
            
            for row in range(self.weights[i].shape[0]):
                for col in range(self.weights[i].shape[1]):
                    # Perturb weight
                    weights_plus = [w.copy() for w in self.weights]
                    weights_minus = [w.copy() for w in self.weights]
                    weights_plus[i][row, col] += h
                    weights_minus[i][row, col] -= h
                    
                    # Compute finite difference
                    loss_plus = compute_loss(weights_plus, self.biases)
                    loss_minus = compute_loss(weights_minus, self.biases)
                    grad_w[row, col] = (loss_plus - loss_minus) / (2 * h)
            
            numerical_grad_w.append(grad_w)
        
        # Similar for biases (simplified for brevity)
        for i in range(self.num_layers):
            numerical_grad_b.append(np.zeros_like(self.biases[i]))
        
        return numerical_grad_w, numerical_grad_b

# Test detailed backpropagation
print("DETAILED BACKPROPAGATION IMPLEMENTATION")
print("=" * 45)

# Create a simple 2-layer network for detailed analysis
detailed_net = DetailedBackpropNetwork([3, 2, 1])

# Test on "Go Dolphins!" example
x = features[0]  # [2, 1, 1]
y = labels[0]    # 1

print(f"Testing on: '{texts[0]}'")
print(f"Features: {x}")
print(f"True label: {y}")

# Forward pass
output, activations, z_values = detailed_net.detailed_forward_pass(x)

# Backward pass
grad_w, grad_b, loss = detailed_net.detailed_backward_pass(x, y, activations, z_values)

print(f"\nFinal Results:")
print(f"Loss: {loss:.6f}")
print(f"Output: {output:.6f}")

print("\n✅ Detailed backpropagation implementation complete!")

In [ ]:
# TODO: Verify backpropagation with numerical gradients
def verify_backpropagation():
    """
    Verify analytical gradients against numerical gradients.
    """
    print("\nBACKPROPAGATION VERIFICATION")
    print("=" * 35)
    
    # Use a smaller network for verification
    verify_net = DetailedBackpropNetwork([3, 2, 1])
    
    x = features[0]
    y = labels[0]
    
    # Analytical gradients
    output, activations, z_values = verify_net.detailed_forward_pass(x, verbose=False)
    analytical_grad_w, analytical_grad_b, _ = verify_net.detailed_backward_pass(
        x, y, activations, z_values, verbose=False
    )
    
    # Numerical gradients (for first layer only due to computational cost)
    print("Computing numerical gradients (first layer only)...")
    numerical_grad_w, numerical_grad_b = verify_net.numerical_gradients(x, y)
    
    # Compare gradients
    print("\nGRADIENT VERIFICATION RESULTS:")
    for i in range(min(2, len(analytical_grad_w))):  # Verify first 2 layers
        print(f"\nLayer {i+1} Weight Gradients:")
        print(f"Analytical:\n{analytical_grad_w[i]}")
        if i < len(numerical_grad_w):
            print(f"Numerical:\n{numerical_grad_w[i]}")
            
            max_diff = np.max(np.abs(analytical_grad_w[i] - numerical_grad_w[i]))
            print(f"Max difference: {max_diff:.2e}")
            
            if max_diff < 1e-6:
                print("✅ Gradients match perfectly!")
            elif max_diff < 1e-4:
                print("✅ Gradients match well")
            else:
                print("⚠️  Gradient mismatch - check implementation")
    
    # Summary of chain rule application
    print("\n" + "="*50)
    print("CHAIN RULE SUMMARY FOR DEEP LEARNING")
    print("="*50)
    print("\n1. FORWARD PASS: Compute activations layer by layer")
    print("   a⁽⁰⁾ = x (input)")
    print("   z⁽ˡ⁾ = W⁽ˡ⁾ᵀa⁽ˡ⁻¹⁾ + b⁽ˡ⁾")
    print("   a⁽ˡ⁾ = σ(z⁽ˡ⁾)")
    print("\n2. BACKWARD PASS: Apply chain rule layer by layer")
    print("   δ⁽ᴸ⁾ = ∂L/∂z⁽ᴸ⁾ (output layer)")
    print("   δ⁽ˡ⁾ = (W⁽ˡ⁺¹⁾δ⁽ˡ⁺¹⁾) ⊙ σ'(z⁽ˡ⁾) (hidden layers)")
    print("\n3. GRADIENTS: Compute parameter updates")
    print("   ∂L/∂W⁽ˡ⁾ = a⁽ˡ⁻¹⁾ ⊗ δ⁽ˡ⁾")
    print("   ∂L/∂b⁽ˡ⁾ = δ⁽ˡ⁾")
    print("\n4. MATHEMATICAL INSIGHT:")
    print("   The chain rule enables us to decompose complex gradients")
    print("   into manageable, local computations at each layer.")
    print("   This is the mathematical foundation of ALL deep learning!")

verify_backpropagation()
print("\n✅ Backpropagation verification complete!")

## What's Next?

You've now mastered the mathematical foundation of deep learning - the chain rule and backpropagation! Here's what we discovered:

**🔑 Key Mathematical Insights:**
1. **Chain Rule Decomposition** - Complex gradients break down into local, manageable computations
2. **Backpropagation Algorithm** - Systematic application of chain rule working backwards through layers
3. **Gradient Flow Dynamics** - How gradients can vanish or explode through deep architectures
4. **Computational Efficiency** - Reusing forward pass computations makes backprop efficient

**🧮 Mathematical Tools Mastered:**
- **Multi-variable chain rule** for composite functions
- **Matrix calculus** for layer-wise gradient computation
- **Numerical verification** of analytical gradients
- **Gradient flow analysis** through deep architectures

**🎯 Why This Matters:**
This analysis reveals the mathematical elegance of deep learning:
- Every deep network uses the same chain rule principle
- Gradient problems (vanishing/exploding) have mathematical explanations
- Activation function choice affects gradient flow properties
- The backpropagation algorithm is just systematic calculus!

**🚀 Coming in Problem 3: Jacobian Analysis**
- How do we analyze sensitivity in multi-output systems?
- What are Jacobian matrices and why do they matter?
- How do we understand system-wide parameter sensitivity?
- What role do Jacobians play in optimization landscapes?

You're building towards a complete mathematical understanding of modern AI! 🐬➡️📊➡️🎯➡️⚡➡️🚀➡️🧮➡️🔗➡️📐